In [1]:
import pandas as pd
import numpy as np
import  pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
import string
from category_encoders import TargetEncoder,LeaveOneOutEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import FeatureHasher
import tensorflow as tf
import xgboost as xgb

#  Get Data

In [2]:

train=pd.read_csv('../input/cat-in-the-dat/train.csv',index_col='id')
test=pd.read_csv('../input/cat-in-the-dat/test.csv',index_col='id')
sub=pd.read_csv('../input/cat-in-the-dat/sample_submission.csv')

In [3]:
ind=len(train)
y=train.target
train.drop('target',axis=1,inplace=True)
#all_data=pd.concat([train,test])

In [4]:
y.value_counts()/len(train)*100

0    69.412
1    30.588
Name: target, dtype: float64

# Encoding categorical features

In [5]:
all_data=pd.concat([train,test])

# Ordinal encoding

In [6]:
ord1_dic={'Grandmaster':4, 'Expert':3, 'Novice':0, 'Contributor':1, 'Master':2}
ord2_dic={'Cold':1, 'Hot':3, 'Lava Hot':5, 'Boiling Hot':4, 'Freezing':0, 'Warm':2}

all_data['ord_1']= all_data.ord_1.map(ord1_dic)
all_data['ord_2']= all_data.ord_2.map(ord2_dic)

In [7]:
enc = OrdinalEncoder()
a=list(string.ascii_lowercase)
b=list(string.ascii_uppercase)
scal= StandardScaler()
all_data[["ord_3"]] = OrdinalEncoder(categories=[a]).fit_transform(all_data[["ord_3"]])
all_data[["ord_4"]] = OrdinalEncoder(categories=[b]).fit_transform(all_data[["ord_4"]])
all_data[["ord_5"]] = OrdinalEncoder().fit_transform(all_data[["ord_5"]])

# nominal encoding

 **<15 CARDINALTY**

**OneHotEncoder**

In [8]:
all_data=pd.get_dummies(all_data,columns=['nom_0','nom_1','nom_2','nom_3','nom_4'],drop_first='True')

**HIHGER THAN 15 CARDINALTY**

# 1-Hash encoding
uses a hash function to map each categorical value in the variable to a unique random number. It’s similar to one-hot encoding but more flexible as it allows us to determine the number of dimension we want to output. The advantage of the hash encoding is lower number of dimensions. However, it can lead to collision of hash numbers in cases where we have large number of categorical features.

In [9]:
nom5_9 =['nom_5','nom_6','nom_7','nom_8','nom_9']
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(input_type='string')
data_h=h.fit_transform(all_data[nom5_9])

#  **2-Frequency Encoding**  
 we count the number of observation for each category and replace the specific value with its equivalent number of occurrence

In [10]:
def fec_enc(df,cols):
    for col in cols:
        n=df.groupby(col).size()/len(df)
        df[col+'en']=df[col].map(n)
    return df
        

In [11]:
fec_enc(all_data,['nom_5','nom_6','nom_7','nom_8','nom_9'])

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_5,nom_6,nom_7,nom_8,nom_9,...,nom_3_India,nom_3_Russia,nom_4_Oboe,nom_4_Piano,nom_4_Theremin,nom_5en,nom_6en,nom_7en,nom_8en,nom_9en
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,...,0,0,0,0,0,0.008624,0.003752,0.000820,0.000978,0.000064
1,0,1,0,T,Y,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,...,0,1,0,1,0,0.002608,0.002736,0.000924,0.000358,0.000046
2,0,0,0,F,Y,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,...,0,1,0,0,1,0.008518,0.003868,0.001600,0.000878,0.000100
3,0,1,0,F,Y,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,...,0,0,1,0,0,0.003256,0.003874,0.001500,0.000714,0.000146
4,0,0,0,F,N,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,...,0,0,1,0,0,0.006602,0.003858,0.001520,0.000690,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,F,N,6c90f81cd,a406a5f12,d02a6b0ba,9e4b23160,acc31291f,...,0,0,0,0,1,0.008912,0.003658,0.001074,0.000522,0.000124
499996,1,0,0,F,Y,2d61990e2,520806ce2,d2d8eabdb,cfbd87ed0,eae3446d0,...,0,0,0,1,0,0.005990,0.002240,0.000824,0.000546,0.000150
499997,0,1,1,T,Y,488406659,28645754b,22831fffe,1108bcd6c,33dd3cf4b,...,0,0,1,0,0,0.002244,0.001216,0.001548,0.000384,0.000040


# 3- Target encoder
 by computing the average of the target variable for each value of the categorical variable and replacing the calculated mean with the categorical value. 

In [12]:
nom5_9=['nom_5','nom_6','nom_7','nom_8','nom_9']
encoder = TargetEncoder()
encoder.fit(train[nom5_9],y)
train[nom5_9]=encoder.transform(train[nom5_9])
test[nom5_9]=encoder.transform(test[nom5_9])

# 4- LeaveOneOut  Encoder

In [13]:
encoder = LeaveOneOutEncoder(return_df=True)
encoder.fit(train[nom5_9],y)
train[nom5_9]=encoder.transform(train[nom5_9])
test[nom5_9]=encoder.transform(test[nom5_9])

# **cyclical features**

# FOR TIME

In [14]:
def cyc_en(df,time,num):
 
    df['sin'+ time] = np.sin(2*np.pi*df[time]/num)
    df['cos'+ time] = np.cos(2*np.pi*df[time]/num)
    return df

In [15]:
cyc_en(all_data,'day',7)
cyc_en(all_data,'month',12)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_5,nom_6,nom_7,nom_8,nom_9,...,nom_4_Theremin,nom_5en,nom_6en,nom_7en,nom_8en,nom_9en,sinday,cosday,sinmonth,cosmonth
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,...,0,0.008624,0.003752,0.000820,0.000978,0.000064,9.749279e-01,-0.222521,0.866025,5.000000e-01
1,0,1,0,T,Y,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,...,0,0.002608,0.002736,0.000924,0.000358,0.000046,-2.449294e-16,1.000000,-0.866025,-5.000000e-01
2,0,0,0,F,Y,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,...,1,0.008518,0.003868,0.001600,0.000878,0.000100,-2.449294e-16,1.000000,0.866025,5.000000e-01
3,0,1,0,F,Y,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,...,0,0.003256,0.003874,0.001500,0.000714,0.000146,9.749279e-01,-0.222521,0.500000,8.660254e-01
4,0,0,0,F,N,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,...,0,0.006602,0.003858,0.001520,0.000690,0.000126,-2.449294e-16,1.000000,-0.866025,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,F,N,6c90f81cd,a406a5f12,d02a6b0ba,9e4b23160,acc31291f,...,1,0.008912,0.003658,0.001074,0.000522,0.000124,7.818315e-01,0.623490,1.000000,6.123234e-17
499996,1,0,0,F,Y,2d61990e2,520806ce2,d2d8eabdb,cfbd87ed0,eae3446d0,...,0,0.005990,0.002240,0.000824,0.000546,0.000150,9.749279e-01,-0.222521,0.866025,5.000000e-01
499997,0,1,1,T,Y,488406659,28645754b,22831fffe,1108bcd6c,33dd3cf4b,...,0,0.002244,0.001216,0.001548,0.000384,0.000040,4.338837e-01,-0.900969,0.500000,8.660254e-01


In [16]:
nom5_9=['nom_5','nom_6','nom_7','nom_8','nom_9']
ohe= OneHotEncoder(sparse=True)


all_data_en= ohe.fit_transform(all_data[nom5_9])

In [17]:
scal= StandardScaler()
all_data=scal.fit_transform(all_data)

ValueError: could not convert string to float: 'T'

# **5-SPARSE MATRIX**

In [ ]:
from scipy.sparse import csr_matrix,hstack
all_data=csr_matrix(all_data)
all_data_en=csr_matrix(all_data_en)

In [ ]:
all_data=hstack([all_data,all_data_en],format='csr')

In [ ]:
new_train=(all_data[:ind])
new_test=(all_data[ind:])